<a href="https://colab.research.google.com/github/gaveter/Project_cs/blob/master/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Project 2019-20
1. Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.
2. For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time.
3. Find the lenders that have funded at least twice.
4. For each country, compute how many loans have involved that country as borrowers.
5. For each country, compute the overall amount of money borrowed.
6. Like the previous point, but expressed as a percentage of the overall amount lent.
7. Like the three previous points, but split for each year (with respect to disburse time).
8. For each lender, compute the overall amount of money lent.
9. For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed.
10. Which country has the highest ratio between the difference computed at the previous point and the population?
11. Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?
12. For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must 
have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016,
 planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04
for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.
© 2019 GitHub, Inc.

## Load libraries

In [0]:
import pandas as pd
from datetime import datetime
import re
import numpy as np

## Task1

### Load csv

In [0]:
loans_lender = pd.read_csv('/content/drive/My Drive/Foundation of computer science/additional-kiva-snapshot/loans_lenders.csv')
len(loans_lender)

In [0]:
loans_lender["lenders"]=loans_lender["lenders"].str.split(',') #divisione lenders
loans_lender_norm=loans_lender.explode('lenders')

### Result

In [5]:
loans_lender_norm=loans_lender_norm.reset_index(drop=True)
loans_lender_norm.head()

,loan_id,lenders
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499


## Task2
- For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time -

In [6]:
loan = pd.read_csv('/content/drive/My Drive/Foundation of computer science/additional-kiva-snapshot/loans.csv')
len(loan)

1419607

In [7]:
loan.dtypes

loan_id                              int64
loan_name                           object
original_language                   object
description                         object
description_translated              object
funded_amount                      float64
loan_amount                        float64
status                              object
activity_name                       object
sector_name                         object
loan_use                            object
country_code                        object
country_name                        object
town_name                           object
currency_policy                     object
currency_exchange_coverage_rate    float64
currency                            object
partner_id                         float64
posted_time                         object
planned_expiration_time             object
disburse_time                       object
raised_time                         object
lender_term                        float64
num_lenders

### Check missing values

In [0]:
missing_loan = pd.isnull(loan[["planned_expiration_time"]])

In [9]:
loan.loc[64:69][["planned_expiration_time","disburse_time"]]

,planned_expiration_time,disburse_time
64,NaN,2010-11-18 08:00:00.000 +0000
65,NaN,2010-11-25 08:00:00.000 +0000
66,NaN,2010-11-25 08:00:00.000 +0000
67,NaN,2010-11-16 08:00:00.000 +0000
68,2016-11-03 06:10:05.000 +0000,2016-09-07 07:00:00.000 +0000
69,NaN,2010-11-10 08:00:00.000 +0000


### Conversion attribute 'planned_expiration_time', 'disburse_time' to string

In [0]:
loan["planned_expiration_time"] = loan.apply(lambda x: str(x["planned_expiration_time"]),axis=1) #each row (value is float)
loan["disburse_time"] = loan.apply(lambda x: str(x["disburse_time"]),axis=1)

In [0]:
def difference_dates(date1,date2):
  if (date1=='nan' or date2=='nan'):
    return(np.nan) #float
  date1 = date1[:-10]
  date1 = datetime.strptime(date1,"%Y-%m-%d %H:%M:%S")
  date2 = date2[:-10]
  date2 = datetime.strptime(date2,"%Y-%m-%d %H:%M:%S")
  diff = (date1 - date2).days
  return(diff)

In [0]:
loan["duration"] = loan.apply(lambda x: difference_dates(x["planned_expiration_time"],x["disburse_time"]),axis=1)

In [15]:
zeroDuration = loan[loan["duration"]==0][["planned_expiration_time","disburse_time","duration"]]
len(zeroDuration)

772

### Result

In [20]:
loan.head()[["planned_expiration_time","disburse_time","duration"]]

,planned_expiration_time,disburse_time,duration
0,2014-02-14 03:30:06.000 +0000,2013-12-22 08:00:00.000 +0000,53.0
1,2014-03-26 22:25:07.000 +0000,2013-12-20 08:00:00.000 +0000,96.0
2,2014-02-15 21:10:05.000 +0000,2014-01-09 08:00:00.000 +0000,37.0
3,2014-02-21 03:10:02.000 +0000,2014-01-17 08:00:00.000 +0000,34.0
4,2014-02-13 06:10:02.000 +0000,2013-12-17 08:00:00.000 +0000,57.0
